# Praktiese voorbeeld met regte data

## In hierdie voorbeeld gaan ons werklike data aflaai (*Data versameling*) en dit skoonmaak (*Ontdek en verstaan data* & *Data voorbereiding*) vir ’n toekomstige toepassing.

## Die data is afkomstig van Statistiek Suid-Afrika en verteenwoordig die Verbruikersprysindeks (VPI) se Gemiddelde Pryse vir alle stedelike gebiede, vanaf Januarie 2017 tot Januarie 2025. Die oorspronklike dataskakel is beskikbaar by: [https://www.statssa.gov.za/?page_id=1847](https://www.statssa.gov.za/?page_id=1847) — kyk spesifiek vir die lêer **CPI_Average Prices_All urban(202501).zip**.

#### *(Die lêer is reeds afgelaai en uitgepak na die klas se GitHub-bladsy vir gebruiksgemak.)*

## Die datastel bevat verskeie datakwaliteitskwessies (jy kan self die oorspronklike aflaai en ondersoek), en ons sal toepaslike datatransformasies uitvoer. Ons skep ook ’n nuwe kenmerk, naamlik *inflasie*, wat gebruik gaan word om ontbrekende waardes in te vul.



In [ ]:
# Laai CSV data lêer af vanaf Github bladsy
!wget https://raw.githubusercontent.com/aby-akademia/NGRDA150-2025/refs/heads/main/datastelle/CPI_Average_Prices_All_urban_202503.csv


In [ ]:
# Laai Pandas biblioteek
import pandas as pd

In [ ]:
# Lees die data vanaf die CSV lêer in 'n DataFrame in, en behandel '..' as ontbrekende waardes
df = pd.read_csv("CPI_Average_Prices_All_urban_202503.csv", na_values=[".."])

# druk die eerste 5 lyne uit na die skerm
df.head()

In [ ]:
# Hernoem met Afrikaanse etikette
afrikaans_etikette = {
    'H04': 'Produk',
    'H07': 'Kode',
    'H08': 'Meeteenheid'
}

df.rename(columns=afrikaans_etikette, inplace=True)

# druk die eerste 5 lyne uit na die skerm
df.head()

In [ ]:
# Standaardiseer die waardes in H06 kolom. Party is "monthly" en ander is "Monthly"
# Gebruik 'str.title()' om net die eerste letter van elke woord in 'n string kapitaliseer.

print(f"Voor standardiseering: {df['H06'].unique()}")

df['H06'] = df['H06'].str.title()

print(f"Na standaardiseering: {df['H06'].unique()}")

In [ ]:
# Identifiseer kolomme wat waarskynlik numeriese pryse bevat (begin met 'M20' of bevat 'Prys')
prys_kolomme = [kolom for kolom in df.columns if kolom.startswith('M20') or 'Prys' in kolom]

# Vervang kommas met punte in desimale getalle en omskep na numeries
# Party waardes in die datastel is bv. 37.89 en ander 37,89
for kolom in prys_kolomme:
    df[kolom] = df[kolom].astype(str).str.replace(',', '.', regex=False)
    df[kolom] = pd.to_numeric(df[kolom], errors='coerce')  # Omskep na float, fout = NaN

# wys eerste paar rye vir kontrole
print(df.head())

In [ ]:
# Standardiseer die datumkolomme (2024-12 in plaas van M202412)
# Hernoem kolomme soos 'M202412' na '2024-12'

datum_kolomme = []
for kol in df.columns:
    if kol.startswith('M20') and len(kol) == 7:
        try:
            jaar = int(kol[1:5])
            maand = int(kol[5:7])
            nuwe_naam = pd.to_datetime(f'{jaar}-{maand}', format='%Y-%m').strftime('%Y-%m')
            df.rename(columns={kol: nuwe_naam}, inplace=True)
            datum_kolomme.append(nuwe_naam)
        except ValueError:
            print(f"Kolom {kol} word oorgeslaan.")

datum_kolomme.sort()

# Druk die eerste paar rye van die DataFrame uit om die veranderinge te sien
print(df.head())

In [ ]:
# Kontroleer op ontbrekende waardes
# Hier tel ons net eers hoeveel rye daar is met ontbrekende waardes in die maand kollome,
# m.a.w. daar is nie pryse vir die produkte vir daardie maande nie.
voor_nulls = df[datum_kolomme].isnull().any(axis=1)
print("Aantal rye met ontbrekende pryse VOOR aanvulling:", voor_nulls.sum())


In [ ]:
# Die metodologie wat ons gaan volg, is soos volg:
# 1. Ons verwyder alle ontbrekende pryse.
# 2. Ons kontroleer dat ’n ry meer as een prys het, indien dit nie die geval is nie, kan ons geen berekeninge daarop doen nie, en ons gaan eenvoudig aan na die volgende ry.
# 3. Ons identifiseer die eerste beskikbare prys saam met die betrokke jaar en maand, en doen dieselfde vir die laaste beskikbare prys.
#    Met hierdie inligting bereken ons die prysverandering oor ’n tydsverloop, wat ons dan gebruik om die inflasie vir daardie produk te skat.
# 4. Ons vul die ontbrekende waardes aan deur gebruik te maak van die berekende inflasie vir die spesifieke produk.

# Skeiding van metadata en tydreeksdata
metadata = df.drop(columns=datum_kolomme)
prysdata = df[datum_kolomme]
aangevulde_prysdata = prysdata.copy()

# Aanvulling per ry gebaseer op inflasie-aanname
for indeks, ry in prysdata.iterrows():
    beskikbare_pryse = ry.dropna()

    if len(beskikbare_pryse) < 2:
        continue  # Onvoldoende data om inflasie te skat

    eerste_datum = beskikbare_pryse.index[0]
    laaste_datum = beskikbare_pryse.index[-1]
    eerste_prys = beskikbare_pryse.iloc[0]
    laaste_prys = beskikbare_pryse.iloc[-1]

    aantal_maande = pd.to_datetime(laaste_datum).to_period("M").ordinal - \
                    pd.to_datetime(eerste_datum).to_period("M").ordinal

    if aantal_maande == 0:
        continue

    inflasiekoers = (laaste_prys / eerste_prys) ** (1 / aantal_maande)

    # Vul slegs die ontbrekende waardes in
    huidige_prys = eerste_prys
    huidige_datum = pd.to_datetime(eerste_datum).to_period("M").ordinal

    # Vul die hele reeks in
    huidige_prys = eerste_prys
    huidige_datum = pd.to_datetime(eerste_datum).to_period("M").ordinal

    for kol in datum_kolomme:
        # Maak seker dat die huidige waarde in aangevulde_prysdata ontbreke is,
        # slegs dan vul ons die berekende waarde in.
        if pd.isna(aangevulde_prysdata.at[indeks, kol]):
            kol_ordinal = pd.to_datetime(kol).to_period("M").ordinal
            maand_verskil = kol_ordinal - huidige_datum
            nuwe_prys = eerste_prys * (inflasiekoers ** maand_verskil)
            aangevulde_prysdata.at[indeks, kol] = nuwe_prys

# Herbou finale stel
volledige_data = pd.concat([metadata, aangevulde_prysdata], axis=1)


In [ ]:
# Finale kontrole van ontbrekende waardes
na_nulls = volledige_data[datum_kolomme].isnull().any(axis=1)
print("Aantal rye met ontbrekende pryse NA aanvulling:", na_nulls.sum())


## Nou plot ons die pryse van die "Maize meal" produk ("Kode" is 10931) voor die imputasie.

In [ ]:
# Laai die Matplotlib biblioteek om te gebruik om te plot
import matplotlib.pyplot as plt

# Maak seker dat die "Kode" kollom hateer word as 'n heelgetal en nie 'n string nie
df['Kode'] = pd.to_numeric(df['Kode'], errors='coerce')

# Kry die "Maize meal" produk ry (in die oorspronklike datastel, df), met Kode 10931
ry_10931 = df[df['Kode'] == 10931]

# Kry die datums
pryse = ry_10931[datum_kolomme].iloc[0]

# Skakel '..' en ander ongeldige waardes om na NaN, en dan na 'float'
pryse_skoon = pd.to_numeric(pryse, errors='coerce')

plt.figure(figsize=(12, 6))
plt.plot(datum_kolomme, pryse_skoon.values, marker='o', linestyle='-')
plt.title('Pryse Oor Tyd vir Produk Kode 10931 (Oorspronklike)')
plt.xlabel('Maand')
plt.ylabel('Prys (Rand)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Ons kan dit dan vergelyk met die pryse van die "Maize meal" produk ("Kode" is 10931) **NA** die imputasie.

In [ ]:
# Ons het klaar Matplotlib hierbo gelaai
# import matplotlib.pyplot as plt

# Maak seker dat die "Kode" kollom hateer word as 'n heelgetal en nie 'n string nie
volledige_data['Kode'] = pd.to_numeric(volledige_data['Kode'], errors='coerce')

# Kry die "Maize meal" produk ry (in die verwerkte datastel, volledige_data), met Kode 10931
ry_10931 = volledige_data[volledige_data['Kode'] == 10931]

# Kry die datums
pryse = ry_10931[datum_kolomme].iloc[0]

# Skakel '..' en ander ongeldige waardes om na NaN, en dan na 'float'
pryse_skoon = pd.to_numeric(pryse, errors='coerce')

plt.figure(figsize=(12, 6))
plt.plot(datum_kolomme, pryse_skoon.values, marker='o', linestyle='-')
plt.title('Pryse Oor Tyd vir Produk Kode 10931 (Verwerkte)')
plt.xlabel('Maand')
plt.ylabel('Prys (Rand)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()